In [3]:
import fiona
from shapely.geometry import shape, mapping, MultiPolygon
from shapely.ops import cascaded_union


fp = 'cantons.geojson'

# Each canton maps to itself
canton_mapping = {
    'Zürich': 'Zürich',
    'Bern/Berne': 'Bern/Berne',
    'Uri': 'Uri',
    'Schwyz': 'Schwyz',
    'Glarus': 'Glarus',
    'Zug': 'Zug',
    'Fribourg': 'Fribourg',
    'Solothurn': 'Solothurn',
    'Basel-Stadt': 'Basel-Stadt',
    'Basel-Landschaft': 'Basel-Landschaft',
    'Schaffhausen': 'Schaffhausen',
    'Appenzell Ausserrhoden': 'Appenzell Ausserrhoden',
    'Appenzell Innerrhoden': 'Appenzell Innerrhoden',
    'St. Gallen': 'St. Gallen',
    'Graubünden/Grigioni': 'Graubünden/Grigioni',
    'Aargau': 'Aargau',
    'Thurgau': 'Thurgau',
    'Ticino': 'Ticino',
    'Vaud': 'Vaud',
    'Valais/Wallis': 'Valais/Wallis',
    'Neuchâtel': 'Neuchâtel',
    'Genève': 'Genève',
    'Jura': 'Jura',
    'Luzern': 'Luzern',
    'Obwalden': 'Obwalden',
    'Nidwalden': 'Nidwalden',
}

# Create a results list for each individual canton
results = [{'id': idx + 1, 'name': canton} for idx, canton in enumerate(canton_mapping.keys())]

results

[{'id': 1, 'name': 'Zürich'},
 {'id': 2, 'name': 'Bern/Berne'},
 {'id': 3, 'name': 'Uri'},
 {'id': 4, 'name': 'Schwyz'},
 {'id': 5, 'name': 'Glarus'},
 {'id': 6, 'name': 'Zug'},
 {'id': 7, 'name': 'Fribourg'},
 {'id': 8, 'name': 'Solothurn'},
 {'id': 9, 'name': 'Basel-Stadt'},
 {'id': 10, 'name': 'Basel-Landschaft'},
 {'id': 11, 'name': 'Schaffhausen'},
 {'id': 12, 'name': 'Appenzell Ausserrhoden'},
 {'id': 13, 'name': 'Appenzell Innerrhoden'},
 {'id': 14, 'name': 'St. Gallen'},
 {'id': 15, 'name': 'Graubünden/Grigioni'},
 {'id': 16, 'name': 'Aargau'},
 {'id': 17, 'name': 'Thurgau'},
 {'id': 18, 'name': 'Ticino'},
 {'id': 19, 'name': 'Vaud'},
 {'id': 20, 'name': 'Valais/Wallis'},
 {'id': 21, 'name': 'Neuchâtel'},
 {'id': 22, 'name': 'Genève'},
 {'id': 23, 'name': 'Jura'},
 {'id': 24, 'name': 'Luzern'},
 {'id': 25, 'name': 'Obwalden'},
 {'id': 26, 'name': 'Nidwalden'}]

In [4]:
def to_shapely(data):
    return shape(data['geometry'])

def to_fiona(data):
    return mapping(data)

def union(target):
    shapes = []
    with fiona.open(fp) as src:
        for feat in src:
            if canton_mapping[feat['properties']['name']] == target:
                shapes.append(to_shapely(feat).buffer(0))
    union_result = cascaded_union(shapes)
    # Ensure the result is a MultiPolygon
    if union_result.geom_type == 'Polygon':
        return MultiPolygon([union_result])
    return union_result


def to_file(fp="model_regions.geojson"):
    meta = {
        'crs': {'no_defs': True, 'ellps': 'WGS84', 'datum': 'WGS84', 'proj': 'longlat'},
        'driver': 'GeoJSON',
        'schema': {'geometry': 'MultiPolygon', 'properties': {'name': 'str', 'id': 'int'}}
    }
    with fiona.drivers():
        with fiona.open(fp, 'w', **meta) as sink:
            for dct in results:
                geom = union(dct['name'])
                sink.write({
                    'geometry': to_fiona(geom),
                    'properties': dct
                })
                
to_file()

/var/folders/v1/5hsb0rrj4wjfjsbnzc38bkb80000gn/T/ipykernel_24188/2140810153.py:26: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
/var/folders/v1/5hsb0rrj4wjfjsbnzc38bkb80000gn/T/ipykernel_24188/2140810153.py:13: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  union_result = cascaded_union(shapes)


In [6]:
import geopandas as gpd
import pandas as pd

df_corrected_coords = pd.read_csv('Geodata/charger_data_geocoded.csv')

geometry = gpd.points_from_xy(df_corrected_coords.lon, df_corrected_coords.lat)
stations_gdf = gpd.GeoDataFrame(df_corrected_coords, geometry=geometry)

# Load model regions
regions_gdf = gpd.read_file('model_regions.geojson')

In [8]:
joined_gdf = gpd.sjoin(stations_gdf, regions_gdf, op='within')
joined_gdf

import os
directory = 'Geodata'
if not os.path.exists(directory):
    os.makedirs(directory)

file_path = os.path.join(directory, 'charger_data_with_canton.csv')

joined_gdf.to_csv(file_path, 
                  sep=",", 
                  encoding='utf-8',
                  index=False)

/Users/matis/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/v1/5hsb0rrj4wjfjsbnzc38bkb80000gn/T/ipykernel_24188/96962583.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined_gdf = gpd.sjoin(stations_gdf, regions_gdf, op='within')
